# Import 

In [1]:
import pandas as pd
import re
import numpy as np
from tqdm import tqdm

from fuzzywuzzy import process, fuzz



# Cleaning IndoT5 Result

In [2]:
df = pd.read_csv("output\IndoT5-small-validation-testing-v4.csv")
# df = pd.read_csv("output\IndoT5-base-validation-testing-v4.csv")
df = pd.read_csv("output\IndoT5-bpara-validation-testing-v4.csv")
# df = pd.read_csv("output\IndoT5-large-validation-testing-v4.csv")

for index, row in df.iterrows():
    print(row['IndoT5_result'])

<pad>[ (' "Saya katakan nanti siapa yang mau masuk kabinet siapa yang mau menjadi menteri saya harus setuju anak-anak harus diberi makan siang kalau kau tidak setuju nggak usah gabung di kabinet Prabowo Subianto," kata Prabowo dalam sambutannya di hadapan puluhan ribu pendukungnya di Stadion GBK, Senayan, Jakarta, Sabtu (10/2/2024).' ;'Prabowo' ;'Pihak yang tidak setuju' ;'Negatif' ),(' "Yang tidak setuju mungkin sebaiknya belajar lagi yang tidak setuju anak-anak Indonesia dikasih makan siang kebangetan. Yang penting rakyat setuju. Penting atau tidak anak-anak kita dapat makanan dan minuman susu. Maukah mau nggak kalian anak-anakmu kuat pintar dan cerdas mau atau tidak?" ucap Prabowo.' ;'Prabowo' ;'Pihak yang tidak setuju' ;'Negatif' ),(' "Saudara-saudara sekalian yang mengatakan makan untuk anak-anak kita tidak penting. Menurut saya bukan orang yang waras bukan orang yang cinta tanah air saya tidak ragu-ragu Prabowo Gibran tidak ragu-ragu kami akan memberi makan siang untuk anak-anak 

In [3]:
def clean_result(text):
    text = text.strip("<pad><extra_id_0>").strip("</s>").strip("]").strip().replace('“', '"').replace('”', '"').replace('â\x80\x9c', '"')

    if text[0] != ("(") and text[0] != ('"'):
        text = text[1:].strip()

    if text[0] == ('"'):
        text = "(" + text
    
    return text

def extract_parts(text):
    pattern1 = r"\(\s*'.*?'\s*;\s*'.*?'\s*;\s*'.*?'\s*;\s*'.*?'\s*\)"
    pattern2 = r"\(\s*'.*?';\s*'.*?';\s*'.*?';\s*'.*?'\s*\)"
    pattern3 = r"\(\s*'.*?';\s*'.*?'\s*\)"


    text_w_pattern1 = re.findall(pattern1, text)
    text_w_pattern2 = re.findall(pattern2, text)
    text_w_pattern3 = re.findall(pattern3, text)

    extracted_parts = []

    for part1 in text_w_pattern1:
        if len(part1.split(";")) == 4:
            extracted_parts.append(part1)
    
    for part2 in text_w_pattern2:
        if len(part2.split(";")) == 3:
            extracted_parts.append(part2)

    for part3 in text_w_pattern3:
        if len(part3.split(";")) == 2:
            extracted_parts.append(part3)

    return extracted_parts

def element_atribution(df):
    df_copy = pd.DataFrame(
        columns=[
            "text",
            "quotation",
            "speaker",
            "target",
            "sentiment",
        ]
    )

    keyword = {
        "quoatation": '"',
        "sentiment" : ["positif", "negatif", "neutral"],
    }

    for _, row in df.iterrows():
        quoataion = "NULL"
        speaker = "NULL"
        target = "NULL"
        sentiment = "NULL"

        splitted = row['matches_format'].strip("(").strip(")").split(";")
        for part in splitted:
            part = part.strip().strip("'")
            
            if keyword["quoatation"] in part:
                quoataion = part.strip(".").replace('“', '"').replace('”', '"').strip()
            elif part.lower() in keyword["sentiment"]:
                sentiment = part
            elif speaker == "NULL":
                speaker = part
            elif target == "NULL":
                target = part

        new_row = pd.DataFrame([
            {
                "text": row['text'],
                "quotation": quoataion,
                "speaker": speaker,
                "target": target,
                "sentiment": sentiment,
            }]
        )

        df_copy = pd.concat([df_copy, new_row])

    return df_copy

def find_similarity_score(texts):
    result_text = texts[0]
    similarities = []

    for text in texts[1:]:
        min_length = min(len(result_text), len(text))
        similarity = process.extractOne(result_text, [text[:min_length]], scorer=fuzz.WRatio)
        similarities.append(similarity[1])  
        
    return np.array([similarities])

def find_most_similar_quotation(texts):
    result_text = texts[0]
    similarities = []

    max_score = 0
    max_index = 0

    for i, text in enumerate(texts[1:]):
        similarity = fuzz.WRatio(result_text, text)
        similarities.append(similarity)
        if similarity > max_score and text[1] == result_text[1]:
            max_score = similarity
            max_index = i

    return texts[1:][max_index]

def clean_quotation_result(news, quotation):
    pattern = r'(?=("[^"]*?[,.!?]".*?\.)'
    pattern = r'("[^"]*?[,.!?]".*?\.)\s'

    news = news.replace('“', '"').replace('”', '"').replace('â\x80\x9c', '"')
    matching_quotation = re.findall(pattern, news)
    
    if matching_quotation == []:
        most_similar_quotation = "-"
    else:
        most_similar_quotation = find_most_similar_quotation([quotation] + matching_quotation).strip()

    return most_similar_quotation  

df_result = df.copy()
df_result['cleaned_result'] = df_result['IndoT5_result'].apply(lambda x: clean_result(x))
df_result['matches_format'] = df_result['cleaned_result'].apply(lambda x: extract_parts(x))
df_result['total_matches'] = df_result['matches_format'].apply(lambda x: len(x))

# display(df_result)

df_result = df_result.drop(columns=['Unnamed: 0','label', 'IndoT5_result', 'cleaned_result'])

df_result = df_result.explode('matches_format').reset_index(drop=True).fillna("NULL")
df_result = element_atribution(df_result).reset_index(drop=True)

df_result['quotation_cleaned'] = df_result.apply(lambda x: clean_quotation_result(x['text'], x['quotation']), axis=1)
df_result = df_result.drop_duplicates()

df_result['quotation'] = df_result['quotation_cleaned']
df_result



,text,quotation,speaker,target,sentiment,quotation_cleaned
0,"Laporan Wartawan Tribunnews.com, Igman Ibrahim...","""Saya katakan nanti siapa yang mau masuk kabin...",Prabowo,Pihak yang tidak setuju,Negatif,"""Saya katakan nanti siapa yang mau masuk kabin..."
1,"Laporan Wartawan Tribunnews.com, Igman Ibrahim...","""Yang tidak setuju mungkin sebaiknya belajar l...",Prabowo,Pihak yang tidak setuju,Negatif,"""Yang tidak setuju mungkin sebaiknya belajar l..."
2,"Laporan Wartawan Tribunnews.com, Igman Ibrahim...","""Saya katakan nanti siapa yang mau masuk kabin...",Prabowo,Pihak yang menyatakan program makan siang gratis,Negatif,"""Saya katakan nanti siapa yang mau masuk kabin..."
3,"Laporan Wartawan Tribunnews.com, Fersianus Wak...","""Terima kasih Pak Jokowi karena sampai dengan ...",Ganjar,Jokowi,Positif,"""Terima kasih Pak Jokowi karena sampai dengan ..."
4,"Laporan Wartawan Tribunnews.com, Fersianus Wak...","""Hak demokrasi, hak politik setiap orang. Seti...",Jokowi,menteri,Positif,"""Hak demokrasi, hak politik setiap orang. Seti..."
...,...,...,...,...,...,...
197,Jakarta - Viral sosok bernama Sappe yang curh...,"""Begitu juga dengan saudara-saudara kami para ...",Sappe,Anies Baswedan,Positif,"""Begitu juga dengan saudara-saudara kami para ..."
198,Jakarta - Viral sosok bernama Sappe yang curh...,"""Fokus ke substansinya saja kalau gitu. Nggak ...",Indra Charismiadji,Anies Baswedan,Positif,"""Fokus ke substansinya saja kalau gitu. Nggak ..."
199,Jakarta - Viral sosok bernama Sappe yang curh...,"""Saya ketemu ojol dan pengangguran yang jadi C...",Indra Charismiadji,Anies Baswedan,Positif,"""Saya ketemu ojol dan pengangguran yang jadi C..."
200,Jakarta - Viral sosok bernama Sappe yang curh...,"""Buzzer paslon lain bilang itu gimmick atau sa...",Sappe,Anies Baswedan,Positif,"""Buzzer paslon lain bilang itu gimmick atau sa..."


# Clean Correct Label

In [4]:
df_label = pd.read_csv("output\Validation Dataset Correct Annotation_v4.csv").fillna("NULL")

df_label = df_label.rename(columns={
    "quotation_sentence": "quotation",
})

if 'original_id' in df_label.columns:
    df_label = df_label.drop(columns=['original_id', 'title'])


df_label

,body,quotation,speaker,target,sentiment
0,"Laporan Wartawan Tribunnews.com, Igman Ibrahim...","""Saya katakan nanti siapa yang mau masuk kabin...",Prabowo,Calon Menteri,Negatif
1,"Laporan Wartawan Tribunnews.com, Igman Ibrahim...","""Yang tidak setuju mungkin sebaiknya belajar l...",Prabowo,Penentang Program Makan Siang Gratis,Negatif
2,"Laporan Wartawan Tribunnews.com, Igman Ibrahim...","""Saudara-saudara sekalian yang mengatakan maka...",Prabowo,Penentang Program Makan Siang Gratis,Negatif
3,"Laporan Wartawan Tribunnews.com, Fersianus Wak...","""Terima kasih Pak Jokowi karena sampai dengan ...",Ganjar Pranowo;Cawapres Ganjar,Jokowi,Positif
4,"Laporan Wartawan Tribunnews.com, Fersianus Wak...","""Hak demokrasi, hak politik setiap orang. Seti...",Jokowi,menteri,Neutral
...,...,...,...,...,...
249,"TRIBUNNEWS.COM, JAKARTA - Pernyataan Calon Pre...","""Bagaimana perasaan Mas Prabowo? Soal etik, et...",Anies Baswedan,Prabowo Subianto,Negatif
250,"TRIBUNNEWS.COM, JAKARTA - Pernyataan Calon Pre...",Anies Baswedan menanggapi pertanyaan wartawan ...,Prabowo Subianto,Anies Baswedan,Negatif
251,TRIBUNNEWS.COM - Calon wakil presiden (Cawapre...,"""Sehingga Mas Gibran lolos dengan melanggar et...",Mahfud Md,Uncle Usman,Negatif
252,Suara.com - Geliat politik elite pasca- Pemilu...,"""Semua, semua (akan diundang bertemu),"" kata J...",Jokowi,Semua elite parpol,Neutral


# Find Clossest Pair

In [6]:
df_label = df_label.rename(columns={"quotation": "quotation_label"})
df_label = df_label.rename(columns={"speaker": "speaker_label"})
df_label = df_label.rename(columns={"target": "target_label"})
df_label = df_label.rename(columns={"sentiment": "sentiment_label"})

df_label['quotation_result'] = "NULL"
df_label['speaker_result'] = "NULL"
df_label['target_result'] = "NULL"
df_label['sentiment_result'] = "NULL"

for index_label, row_label in tqdm(df_label.iterrows(), total=df_label.shape[0]):
    for index_result, row_result in df_result.iterrows():
        if row_label['quotation_label'] == row_result['quotation']:
            df_label.at[index_label, 'quotation_result'] = row_result['quotation']
            df_label.at[index_label, 'speaker_result'] = row_result['speaker']
            df_label.at[index_label, 'target_result'] = row_result['target']
            df_label.at[index_label, 'sentiment_result'] = row_result['sentiment']

df_label

  0%|          | 0/254 [00:00<?, ?it/s]

100%|██████████| 254/254 [00:06<00:00, 36.32it/s]


,body,quotation_label,speaker_label,target_label,sentiment_label,quotation_result,speaker_result,target_result,sentiment_result
0,"Laporan Wartawan Tribunnews.com, Igman Ibrahim...","""Saya katakan nanti siapa yang mau masuk kabin...",Prabowo,Calon Menteri,Negatif,"""Saya katakan nanti siapa yang mau masuk kabin...",Prabowo,Pihak yang menyatakan program makan siang gratis,Negatif
1,"Laporan Wartawan Tribunnews.com, Igman Ibrahim...","""Yang tidak setuju mungkin sebaiknya belajar l...",Prabowo,Penentang Program Makan Siang Gratis,Negatif,"""Yang tidak setuju mungkin sebaiknya belajar l...",Prabowo,Pihak yang tidak setuju,Negatif
2,"Laporan Wartawan Tribunnews.com, Igman Ibrahim...","""Saudara-saudara sekalian yang mengatakan maka...",Prabowo,Penentang Program Makan Siang Gratis,Negatif,NULL,NULL,NULL,NULL
3,"Laporan Wartawan Tribunnews.com, Fersianus Wak...","""Terima kasih Pak Jokowi karena sampai dengan ...",Ganjar Pranowo;Cawapres Ganjar,Jokowi,Positif,"""Terima kasih Pak Jokowi karena sampai dengan ...",Ganjar,Jokowi,Positif
4,"Laporan Wartawan Tribunnews.com, Fersianus Wak...","""Hak demokrasi, hak politik setiap orang. Seti...",Jokowi,menteri,Neutral,"""Hak demokrasi, hak politik setiap orang. Seti...",Jokowi,menteri,Positif
...,...,...,...,...,...,...,...,...,...
249,"TRIBUNNEWS.COM, JAKARTA - Pernyataan Calon Pre...","""Bagaimana perasaan Mas Prabowo? Soal etik, et...",Anies Baswedan,Prabowo Subianto,Negatif,"""Bagaimana perasaan Mas Prabowo? Soal etik, et...",Sudirman Said,Pemimpin di sektor publik,Negatif
250,"TRIBUNNEWS.COM, JAKARTA - Pernyataan Calon Pre...",Anies Baswedan menanggapi pertanyaan wartawan ...,Prabowo Subianto,Anies Baswedan,Negatif,NULL,NULL,NULL,NULL
251,TRIBUNNEWS.COM - Calon wakil presiden (Cawapre...,"""Sehingga Mas Gibran lolos dengan melanggar et...",Mahfud Md,Uncle Usman,Negatif,"""Sehingga Mas Gibran lolos dengan melanggar et...",Mahfud Md,Uncle Usman,Negatif
252,Suara.com - Geliat politik elite pasca- Pemilu...,"""Semua, semua (akan diundang bertemu),"" kata J...",Jokowi,Semua elite parpol,Neutral,"""Semua, semua (akan diundang bertemu),"" kata J...",Fahrul Muzaqqi,Surya Paloh,Neutral


# Scoring

In [8]:
def name_similarity_score(target_name, other_names):
    if target_name == "NULL" or other_names == "NULL":
        return 0
    
    similarities = []
    if ";" in other_names:
        other_names = other_names.split(";")
    else:
        other_names = [other_names]

    similarities = process.extractOne(target_name, other_names, scorer=fuzz.WRatio)
    

    return similarities[1]

In [9]:
df_evaluation = df_result.copy()
df_evaluation = df_label.copy()

df_evaluation['quotation_score'] = 0
df_evaluation['speaker_score'] = 0
df_evaluation['target_score'] = 0
df_evaluation['sentiment_score'] = 0

for i, row in df_evaluation.iterrows():
    if row['sentiment_result'] == row['sentiment_label']:
        df_evaluation.at[i, 'sentiment_score'] = 100
        
    if row['quotation_result'] == row['quotation_label']:
        df_evaluation.at[i, 'quotation_score'] = 100

    df_evaluation.at[i, 'speaker_score'] = name_similarity_score(row['speaker_result'], row['speaker_label'])
    df_evaluation.at[i, 'target_score'] = name_similarity_score(row['target_result'], row['target_label'])

df_evaluation = df_evaluation.sort_values(by=['quotation_score', 'speaker_score', 'target_score', 'sentiment_score'], ascending=False)

print("Quotation Accuracy: ", "{:.2f} %".format(df_evaluation['quotation_score'].apply(lambda x: 100 if x >= 80 else 0).mean()))
print("Sentiment Accuracy: ", "{:.2f} %".format(df_evaluation['sentiment_score'].mean()))
print("Speaker Accuracy: ", "{:.2f} %".format(df_evaluation['speaker_score'].apply(lambda x: 100 if x >= 70 else 0).mean()))
print("Target Accuracy: ", "{:.2f} %".format(df_evaluation['target_score'].apply(lambda x: 100 if x >= 70 else 0).mean()))

print()
print("Total Exrtacted Quotatin", df_result.shape[0])
print("Total Correct Quotation", df_label.shape[0])
print()
print("Correct Quotation: ", df_evaluation['quotation_score'].apply(lambda x: 1 if x == 100 else 0).sum())
print("Correct Sentiment: ", df_evaluation['sentiment_score'].apply(lambda x: 1 if x == 100 else 0).sum())
print("Correct Speaker: ", df_evaluation['speaker_score'].apply(lambda x: 1 if x >= 70 else 0).sum())
print("Correct Target: ", df_evaluation['target_score'].apply(lambda x: 1 if x >= 70 else 0).sum())


# display(df_evaluation.head(50))
# df_evaluation.loc[50:]

Quotation Accuracy:  64.96 %
Sentiment Accuracy:  40.55 %
Speaker Accuracy:  59.84 %
Target Accuracy:  38.98 %

Total Exrtacted Quotatin 199
Total Correct Quotation 254

Correct Quotation:  165
Correct Sentiment:  103
Correct Speaker:  152
Correct Target:  99


In [10]:
# print quoatation result and quotation label
for index, row in df_evaluation.iterrows():

    print("Index  : ", index)
    print("Result : ", row['quotation_result'])
    print("Label  : ", row['quotation_label'])
    print("Score  : ", row['quotation_score'])
    print("Sentiment Result : ", row['sentiment_result'])
    print("Sentiment Label  : ", row['sentiment_label'])
    print("="*40)

Index  :  11
Result :  "Saya bertemu dan mengenal Prabowo tahun 1970, tepatnya bulan Januari. Jadi 53 tahun yang lalu. Kami bertemu di kampus Akabri Darat, sekarang Akademi Militer di Lembah Tidar, karena kami sama dengan tekad bulat ingin menjadi prajurit, kami sama-sama ingin menjadi patriot pembela Tanah Air. Setelah menyelesaikan pendidikan di akademi militer, dalam perkembangannya, Prabowo masuk kor baret merah yakni Kopassus, sedangkan saya baret hijau, Kostrad," cerita SBY mengenang perkenalannya dengan Prabowo di depan peserta acara Konsolidasi Pemenangan Capres Prabowo Subianto yang digelar DPP Partai Demokrat di Hotel Aston Madiun, Senin (20/11/2023).
Label  :  "Saya bertemu dan mengenal Prabowo tahun 1970, tepatnya bulan Januari. Jadi 53 tahun yang lalu. Kami bertemu di kampus Akabri Darat, sekarang Akademi Militer di Lembah Tidar, karena kami sama dengan tekad bulat ingin menjadi prajurit, kami sama-sama ingin menjadi patriot pembela Tanah Air. Setelah menyelesaikan pendidi

In [11]:
# print quoatation result and quotation label
for index, row in df_evaluation.sort_values(by=['speaker_score'], ascending=False).iterrows():
    print("Index  : ", index)
    print("Quotation : ", row['quotation_result'])
    print("Result : ", row['speaker_result'])
    print("Label  : ", row['speaker_label'])
    print("Score  : ", row['speaker_score'])
    print("="*40)

Index  :  11
Quotation :  "Saya bertemu dan mengenal Prabowo tahun 1970, tepatnya bulan Januari. Jadi 53 tahun yang lalu. Kami bertemu di kampus Akabri Darat, sekarang Akademi Militer di Lembah Tidar, karena kami sama dengan tekad bulat ingin menjadi prajurit, kami sama-sama ingin menjadi patriot pembela Tanah Air. Setelah menyelesaikan pendidikan di akademi militer, dalam perkembangannya, Prabowo masuk kor baret merah yakni Kopassus, sedangkan saya baret hijau, Kostrad," cerita SBY mengenang perkenalannya dengan Prabowo di depan peserta acara Konsolidasi Pemenangan Capres Prabowo Subianto yang digelar DPP Partai Demokrat di Hotel Aston Madiun, Senin (20/11/2023).
Result :  SBY
Label  :  SBY
Score  :  100
Index  :  185
Quotation :  "Beliau ini termasuk dari awal menjadi bupati di umur 31, udah jadi bupati. Saya dulu jadi wali kota umur 33 34. Beliau ini luar biasa. Dan beliau ini salah satu idola dan salah satu alasan saya masuk politik itu ya beliau anak muda berprestasi dan mau menga

In [12]:
# print quoatation result and quotation label
for index, row in df_evaluation.sort_values(by=['target_score'], ascending=False).iterrows():
    print("Index  : ", index)
    print("Quotation : ", row['quotation_result'])
    print("Target : ", row['target_result'])
    print("Label  : ", row['target_label'])
    print("Score  : ", row['target_score'])
    print("="*40)

Index  :  11
Quotation :  "Saya bertemu dan mengenal Prabowo tahun 1970, tepatnya bulan Januari. Jadi 53 tahun yang lalu. Kami bertemu di kampus Akabri Darat, sekarang Akademi Militer di Lembah Tidar, karena kami sama dengan tekad bulat ingin menjadi prajurit, kami sama-sama ingin menjadi patriot pembela Tanah Air. Setelah menyelesaikan pendidikan di akademi militer, dalam perkembangannya, Prabowo masuk kor baret merah yakni Kopassus, sedangkan saya baret hijau, Kostrad," cerita SBY mengenang perkenalannya dengan Prabowo di depan peserta acara Konsolidasi Pemenangan Capres Prabowo Subianto yang digelar DPP Partai Demokrat di Hotel Aston Madiun, Senin (20/11/2023).
Target :  Prabowo
Label  :  Prabowo
Score  :  100
Index  :  12
Quotation :  "Di situlah kami sering berinteraksi, berpikir bersama, bekerja bersama, karena kami ingin memajukan profesionalitas TNI menjadi tentara modern. Itu yang sering kami diskusikan dan dalam porsi kami masing-masing kami lakukan," lanjutnya.
Target :  Pra

In [13]:
# print quoatation result and quotation label
for index, row in df_evaluation.sort_values(by=['sentiment_score'], ascending=False).iterrows():
    print("Index  : ", index)
    print("Result : ", row['quotation_result'])
    print("Target : ", row['sentiment_result'])
    print("Label  : ", row['sentiment_label'])
    print("Score  : ", row['sentiment_score'])
    print("="*40)

Index  :  11
Result :  "Saya bertemu dan mengenal Prabowo tahun 1970, tepatnya bulan Januari. Jadi 53 tahun yang lalu. Kami bertemu di kampus Akabri Darat, sekarang Akademi Militer di Lembah Tidar, karena kami sama dengan tekad bulat ingin menjadi prajurit, kami sama-sama ingin menjadi patriot pembela Tanah Air. Setelah menyelesaikan pendidikan di akademi militer, dalam perkembangannya, Prabowo masuk kor baret merah yakni Kopassus, sedangkan saya baret hijau, Kostrad," cerita SBY mengenang perkenalannya dengan Prabowo di depan peserta acara Konsolidasi Pemenangan Capres Prabowo Subianto yang digelar DPP Partai Demokrat di Hotel Aston Madiun, Senin (20/11/2023).
Target :  Positif
Label  :  Positif
Score  :  100
Index  :  164
Result :  "Tidak masalah satu putaran. Yang penting itu adalah pilpres berlangsung jurdil tanpa kecurangan. Bisa saja nanti pasangan Anies-Muhaimin yang menang satu putaran kalau melihat gairah rakyat yang begitu besar terhadap perubahan agar harapan baru muncul dan